In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob

# 추정값들을 저장해놓은 csv 파일들의 경로입니다.
paths = sorted(glob("/opt/ml/code/prediction/*.csv"))
num_paths = len(paths) # csv 파일 개수

# hard vote 시, 동점인 경우 점수가 가장 좋았던 데이터에 결정권을 주었습니다.
# - 처음에는 왼쪽의 자연 정렬(숫자 정렬)된 순서를 기준으로 했습니다.
# - 그런데 코드에서는 경로가 문자이기 때문에 문자열 정렬이 되므로 순서가 다름에 주의해야 합니다.
#   아래의 paths를 직접 보고 결정하도록 합니다.
paths

['/opt/ml/code/prediction/submission10_token_and_minor_label.csv',
 '/opt/ml/code/prediction/submission11_markedsent_maxlen204.csv',
 '/opt/ml/code/prediction/submission13_data_aug_less_epoch.csv',
 '/opt/ml/code/prediction/submission14_15epoch_to_13epoch.csv',
 '/opt/ml/code/prediction/submission3_20epochs.csv',
 '/opt/ml/code/prediction/submission4_koelectra.csv']

In [21]:

king = 0 # 동점인 경우에 따를 csv파일의 인덱스입니다.

# csv 파일들을 DataFrame의 각 열에 담습니다. (len_testset-rows x num_file-columns)
# - pd.read_csv(path)
# - csv 파일의 형식이 ","가 아닌 "\t"로 구분되므로 delimiter를 지정해줍니다.
# - 또한 header도 없으므로 None으로 설정해줍니다.
#
# 만약 list 형태로 담으면 전치된 형태가 됩니다. (num_file-rows x len_testset-columns)
# df = pd.DataFrame([
#     pd.read_csv(path, delimiter="\t", header=None)[0][1:] for path in paths
# ])
df = pd.DataFrame({
    i: pd.read_csv(paths[i], delimiter="\t", header=None)[0][1:] for i in range(num_paths)
})
len_testset = len(df[0]) # 1000

output = [] # 투표 결과를 담습니다.
for i in range(len_testset):
    # 각 데이터(행) 별로 예측값과 해당 개수를 저장합니다.
    preds, cnts = [], []
    for pred, cnt in df.iloc[i].value_counts().items():
        preds.append(pred)
        cnts.append(cnt)

    # 예측값이 한 가지이면 만장일치입니다.
    if len(preds) == 1:
        ans = preds[0]
    # 그렇지 않으면, 투표를 시행합니다.
    else:
        i1, i2 = np.argsort(cnts)[:2] # i1, i2는 두 최상위 득표수의 인덱스
        # 최상위 득표수가 동점이면 투표로 결론나지 않습니다.
        # 이 때는 결정권자인 king의 예측값을 선택합니다.
        if cnts[i1] == cnts[i2]:
            ans = df[king][i+1]
        # 최상위 득표수가 하나이면 해당 예측값을 선택합니다.
        else:
            ans = preds[i1]
    output.append(ans)

# DataFrame에 담습니다.
# - 이번 대회의 제출 형식에 따라, columns=["pred"]로 설정합니다.
output = pd.DataFrame(output, columns=["pred"])
#output.head()

# 저장
output_dir = "/opt/ml/code/prediction/ensemble.csv"
output.to_csv(output_dir, index=False)

# 잘 저장되었는지 확인합니다.
pd.read_csv(output_dir, delimiter="\t", header=None)